In [ ]:
#C:\Users\Asus\AppData\Roaming\Python\Python39\Scripts

In [1]:
from sklearn.cluster import AgglomerativeClustering
from pytesseract import Output
from tabulate import tabulate
import pandas as pd
import numpy as np
import pytesseract
import argparse
import imutils
import cv2
from pdf2image import convert_from_path
 
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    

In [ ]:
#Convierte PDF a imagen
# Store Pdf with convert_from_path function
images = convert_from_path('pdfs/Asis-vot-OFICIAL-05-08-21.pdf', poppler_path=r'C:\Users\Asus\Documents\Python\Notebooks\ushnu\votaciones congreso\poppler-0.68.0_x86\poppler-0.68.0\bin')
 
    
    

In [ ]:
for i in range(len(images)):
      # Save pages as images in the pdf
    images[i].save('pdfs/imgs/20210805/'+ str(i) +'.jpg', 'JPEG')

In [ ]:
#en la prueba la votacion es pdfs/imgs/20210805/2

In [12]:
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser(fromfile_prefix_chars='@')
ap.add_argument("-i", "--image", required=True, help="path to input image to be OCR'd")
ap.add_argument("-o", "--output", required=True, help="path to output CSV file")
ap.add_argument("-c", "--min-conf", type=int, default=0, help="minimum confidence value to filter weak text detection")
ap.add_argument("-d", "--dist-thresh", type=float, default=25.0, help="distance threshold cutoff for clustering")
ap.add_argument("-s", "--min-size", type=int, default=2, help="minimum cluster size (i.e., # of entries in column)")
parsed = ap.parse_args("-i pdfs/imgs/20210805/2.jpg -o pdfs/imgs/20210805/text.csv -d 30".split())
args = vars(parsed)

In [13]:
#funcion para ajustar imagen
def ResizeWithAspectRatio(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    return cv2.resize(image, dim, interpolation=inter)

In [14]:
#Codigo para borrar el titulo y el contorno del sello
#Se tienen que borrar porque interfieren al ubicar la tabla

# set a seed for our random number generator
np.random.seed(42)
# load the input image and convert it to grayscale
image = cv2.imread(args["image"])
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# initialize a rectangular kernel that is ~5x wider than it is tall,
# then smooth the image using a 3x3 Gaussian blur and then apply a
# blackhat morphological operator to find dark regions on a light
# background
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (51, 11))
gray = cv2.GaussianBlur(gray, (3, 3), 0)
blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)
# compute the Scharr gradient of the blackhat image and scale the
# result into the range [0, 255]
grad = cv2.Sobel(blackhat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
grad = np.absolute(grad)
(minVal, maxVal) = (np.min(grad), np.max(grad))
grad = (grad - minVal) / (maxVal - minVal)
grad = (grad * 255).astype("uint8")
# apply a closing operation using the rectangular kernel to close
# gaps in between characters, apply Otsu's thresholding method, and
# finally a dilation operation to enlarge foreground regions
grad = cv2.morphologyEx(grad, cv2.MORPH_CLOSE, kernel)
thresh = cv2.threshold(grad, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
thresh = cv2.dilate(thresh, None, iterations=2)
# cv2.imshow("Thresh", ResizeWithAspectRatio(thresh, width=400))

# find contours in the thresholded image and grab the largest one,
# which we will assume is the stats table
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts_area = {cv2.contourArea(cnt):cnt for cnt in cnts}
areas_sorted = sorted(cnts_area.keys())
tableCnt = cnts_area[areas_sorted[-1]]

sello_area = areas_sorted[-1]
selloCnt = cnts_area[sello_area]

titulo_area = areas_sorted[-2]
tituloCnt = cnts_area[titulo_area]

(x_titulo, y_titulo, w_titulo, h_titulo) = cv2.boundingRect(tituloCnt)
(x_sello, y_sello, w_sello, h_sello) = cv2.boundingRect(selloCnt)

#De nuevo sin el titulo y sin el sello
image = image[y_titulo + h_titulo : y_sello, :]
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# initialize a rectangular kernel that is ~5x wider than it is tall,
# then smooth the image using a 3x3 Gaussian blur and then apply a
# blackhat morphological operator to find dark regions on a light
# background
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (51, 11))
gray = cv2.GaussianBlur(gray, (3, 3), 0)
blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)
# compute the Scharr gradient of the blackhat image and scale the
# result into the range [0, 255]
grad = cv2.Sobel(blackhat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
grad = np.absolute(grad)
(minVal, maxVal) = (np.min(grad), np.max(grad))
grad = (grad - minVal) / (maxVal - minVal)
grad = (grad * 255).astype("uint8")
# apply a closing operation using the rectangular kernel to close
# gaps in between characters, apply Otsu's thresholding method, and
# finally a dilation operation to enlarge foreground regions
grad = cv2.morphologyEx(grad, cv2.MORPH_CLOSE, kernel)
thresh = cv2.threshold(grad, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
thresh = cv2.dilate(thresh, None, iterations=10)

# find contours in the thresholded image and grab the largest one,
# which we will assume is the stats table
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts_area = {cv2.contourArea(cnt):cnt for cnt in cnts}
areas_sorted = sorted(cnts_area.keys())
tableCnt = cnts_area[areas_sorted[-1]]

(x, y, w, h) = cv2.boundingRect(tableCnt)
table = image[y:y + h, x:x + w]
# show the original input image and extracted table to our screen
cv2.imshow("Input", ResizeWithAspectRatio(image, width=400))
cv2.imshow("Table", ResizeWithAspectRatio(table, width=400))
cv2.waitKey(0)
cv2.destroyAllWindows()


In [15]:
# set the PSM mode to detect sparse text, and then localize text in
# the table
options = "--psm 6"
results = pytesseract.image_to_data(cv2.cvtColor(table, cv2.COLOR_BGR2RGB), config=options, output_type=Output.DICT)
# initialize a list to store the (x, y)-coordinates of the detected
# text along with the OCR'd text itself
coords = []
ocrText = []

In [16]:
cv2.imshow("Table", cv2.cvtColor(ResizeWithAspectRatio(table, width=900), cv2.COLOR_BGR2RGB) )
cv2.waitKey(0)
cv2.destroyAllWindows()

In [17]:
# loop over each of the individual text localizations
for i in range(0, len(results["text"])):
    # extract the bounding box coordinates of the text region from
    # the current result
    x = results["left"][i]
    y = results["top"][i]
    w = results["width"][i]
    h = results["height"][i]
    # extract the OCR text itself along with the confidence of the
    # text localization
    text = results["text"][i]
    conf = float(results["conf"][i])
    # filter out weak confidence text localizations
    if conf > args["min_conf"]:
        # update our text bounding box coordinates and OCR'd text,
        # respectively
        coords.append((x, y, w, h))
        ocrText.append(text)

In [18]:
# extract all x-coordinates from the text bounding boxes, setting the
# y-coordinate value to zero
xCoords = [(c[0], 0) for c in coords]
# apply hierarchical agglomerative clustering to the coordinates
clustering = AgglomerativeClustering(
	n_clusters=None,
	affinity="manhattan",
	linkage="complete",
	distance_threshold=args["dist_thresh"])
clustering.fit(xCoords)
# initialize our list of sorted clusters
sortedClusters = []

In [19]:
# loop over all clusters
for l in np.unique(clustering.labels_):
	# extract the indexes for the coordinates belonging to the
	# current cluster
	idxs = np.where(clustering.labels_ == l)[0]
	# verify that the cluster is sufficiently large
	if len(idxs) > args["min_size"]:
		# compute the average x-coordinate value of the cluster and
		# update our clusters list with the current label and the
		# average x-coordinate
		avg = np.average([coords[i][0] for i in idxs])
		sortedClusters.append((l, avg))
# sort the clusters by their average x-coordinate and initialize our
# data frame
sortedClusters.sort(key=lambda x: x[1])
df = pd.DataFrame()

In [20]:
# loop over the clusters again, this time in sorted order
for (l, _) in sortedClusters:
	# extract the indexes for the coordinates belonging to the
	# current cluster
	idxs = np.where(clustering.labels_ == l)[0]
	# extract the y-coordinates from the elements in the current
	# cluster, then sort them from top-to-bottom
	yCoords = [coords[i][1] for i in idxs]
	sortedIdxs = idxs[np.argsort(yCoords)]
	# generate a random color for the cluster
	color = np.random.randint(0, 255, size=(3,), dtype="int")
	color = [int(c) for c in color]

	# loop over the sorted indexes
	for i in sortedIdxs:
		# extract the text bounding box coordinates and draw the
		# bounding box surrounding the current element
		(x, y, w, h) = coords[i]
		cv2.rectangle(table, (x, y), (x + w, y + h), color, 2)
	# extract the OCR'd text for the current column, then construct
	# a data frame for the data where the first entry in our column
	# serves as the header
	cols = [ocrText[i].strip() for i in sortedIdxs]
	currentDF = pd.DataFrame({cols[0]: cols[1:]})
	# concatenate *original* data frame with the *current* data
	# frame (we do this to handle columns that may have a varying
	# number of rows)
	df = pd.concat([df, currentDF], axis=1)

In [24]:
# replace NaN values with an empty string and then show a nicely
# formatted version of our multi-column OCR'd text
df.fillna("", inplace=True)
print(tabulate(df, headers="keys", tablefmt="psql"))
# write our table to disk as a CSV file
print("[INFO] saving CSV file to disk...")
df.to_csv(args["output"], index=False)
# show the output image after performing multi-column OCR
cv2.imshow("Output", ResizeWithAspectRatio(image, width=900))
cv2.waitKey(0)
cv2.destroyAllWindows()

+----+--------+------------+------------+-------------+-----------+------------+----------+-------------+-------------+-----------+-----------+------+-------+---------+-------------+-------------+------------+------------+--------------+-----------+---------+-----------+-----------+----------+---------+------+-------+--------+-------------+---------------+------------+--------------+-----------+-----------+-----------+---------+------------+------------+------+-------+
|    | APP    | ACUNA      | PERALTA,   | RECUENCO,   | MARIA     | MARIA      | MARIA    | ALEJANDRO   | GRIMANEZA   | HECTOR    | ANTONIO   | SI   | +++   | SP-PM   | ELERA       | GARCLA,     | ANCACHI,   | VARGAS,    | VICTOR       | WILMAR    | JHAEC   | LUIS      | ALBERTO   | DARWIN   | MARIA   | ST   | +++   | PL     | PAREDES     | RODRIGUEZ,    | QUEDO,     | CASTRO,      | AVALOS,   | SUSEL     | FRANCIS   | KELLY   | IVETT      | JHASMINA   | SI   | +++   |
|----+--------+------------+------------+-----------

In [36]:
# initialize a rectangular kernel that is ~5x wider than it is tall,
# then smooth the image using a 3x3 Gaussian blur and then apply a
# blackhat morphological operator to find dark regions on a light
# background
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (51, 11))
gray = cv2.GaussianBlur(gray, (3, 3), 0)
blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)
# compute the Scharr gradient of the blackhat image and scale the
# result into the range [0, 255]
grad = cv2.Sobel(blackhat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
grad = np.absolute(grad)
(minVal, maxVal) = (np.min(grad), np.max(grad))
grad = (grad - minVal) / (maxVal - minVal)
grad = (grad * 255).astype("uint8")
# apply a closing operation using the rectangular kernel to close
# gaps in between characters, apply Otsu's thresholding method, and
# finally a dilation operation to enlarge foreground regions
grad = cv2.morphologyEx(grad, cv2.MORPH_CLOSE, kernel)
thresh = cv2.threshold(grad, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
thresh = cv2.dilate(thresh, None, iterations=2)
cv2.imshow("Thresh", ResizeWithAspectRatio(thresh, width=400))
cv2.waitKey()
cv2.destroyAllWindows()

In [39]:
# find contours in the thresholded image and grab the largest one,
# which we will assume is the stats table
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
tableCnt = max(cnts, key=cv2.contourArea)
# compute the bounding box coordinates of the stats table and extract
# the table from the input image
(x, y, w, h) = cv2.boundingRect(tableCnt)
table = image[y:y + h, x:x + w]
# show the original input image and extracted table to our screen
cv2.imshow("Input", ResizeWithAspectRatio(image, width=400))
cv2.imshow("Table", ResizeWithAspectRatio(table, width=400))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# set the PSM mode to detect sparse text, and then localize text in
# the table
options = "--psm 6"
results = pytesseract.image_to_data(
	cv2.cvtColor(table, cv2.COLOR_BGR2RGB),
	config=options,
	output_type=Output.DICT)
# initialize a list to store the (x, y)-coordinates of the detected
# text along with the OCR'd text itself
coords = []
ocrText = []

In [ ]:
# loop over each of the individual text localizations
for i in range(0, len(results["text"])):
	# extract the bounding box coordinates of the text region from
	# the current result
	x = results["left"][i]
	y = results["top"][i]
	w = results["width"][i]
	h = results["height"][i]
	# extract the OCR text itself along with the confidence of the
	# text localization
	text = results["text"][i]
	conf = int(results["conf"][i])
	# filter out weak confidence text localizations
	if conf > args["min_conf"]:
		# update our text bounding box coordinates and OCR'd text,
		# respectively
		coords.append((x, y, w, h))
		ocrText.append(text)